In [147]:
import torch
import torch.nn as nn

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns
from sklearn.model_selection import train_test_split
import os
import sys
import time
import warnings
from sklearn.preprocessing import OneHotEncoder
import matplotlib as mpl

import tensorflow as tf
from tensorflow import keras


torch.cuda.is_available()

True

In [148]:
x = torch.rand(5, 3)
print(x)

tensor([[0.8992, 0.9438, 0.7985],
        [0.0964, 0.4051, 0.5221],
        [0.6186, 0.4858, 0.4513],
        [0.2702, 0.6776, 0.2946],
        [0.9766, 0.9294, 0.7464]])


In [149]:

import os.path


train_all = pd.read_csv('./train.csv')
test = pd.read_csv('./test.csv')


train_all.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8693 entries, 0 to 8692
Data columns (total 14 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   PassengerId   8693 non-null   object 
 1   HomePlanet    8492 non-null   object 
 2   CryoSleep     8476 non-null   object 
 3   Cabin         8494 non-null   object 
 4   Destination   8511 non-null   object 
 5   Age           8514 non-null   float64
 6   VIP           8490 non-null   object 
 7   RoomService   8512 non-null   float64
 8   FoodCourt     8510 non-null   float64
 9   ShoppingMall  8485 non-null   float64
 10  Spa           8510 non-null   float64
 11  VRDeck        8505 non-null   float64
 12  Name          8493 non-null   object 
 13  Transported   8693 non-null   bool   
dtypes: bool(1), float64(6), object(7)
memory usage: 891.5+ KB


In [150]:
train, valid = train_test_split(train_all, test_size=0.2, random_state=42)

train_X, train_y = train.drop('Transported', axis=1), train['Transported']
valid_X, valid_y = valid.drop('Transported', axis=1), valid['Transported']
train_X.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 6954 entries, 2333 to 7270
Data columns (total 13 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   PassengerId   6954 non-null   object 
 1   HomePlanet    6786 non-null   object 
 2   CryoSleep     6777 non-null   object 
 3   Cabin         6796 non-null   object 
 4   Destination   6815 non-null   object 
 5   Age           6806 non-null   float64
 6   VIP           6792 non-null   object 
 7   RoomService   6828 non-null   float64
 8   FoodCourt     6814 non-null   float64
 9   ShoppingMall  6789 non-null   float64
 10  Spa           6820 non-null   float64
 11  VRDeck        6803 non-null   float64
 12  Name          6795 non-null   object 
dtypes: float64(6), object(7)
memory usage: 760.6+ KB


In [151]:
train_X["PassengerId"]

2333    2513_01
2589    2774_02
8302    8862_04
8177    8736_02
500     0539_02
         ...   
5734    6076_01
5191    5537_01
5390    5756_06
860     0925_01
7270    7775_01
Name: PassengerId, Length: 6954, dtype: object

In [152]:

train_X["relation"] = train_X["PassengerId"].str.split("_").str[1].astype(int)
train_X["relation"]

2333    1
2589    2
8302    4
8177    2
500     2
       ..
5734    1
5191    1
5390    6
860     1
7270    1
Name: relation, Length: 6954, dtype: int64

In [153]:
cat_encoder = OneHotEncoder()
housing_cat_encoded  = cat_encoder.fit_transform(train_X[["HomePlanet"]])
housing_cat_encoded.toarray()
train_X["HomePlanet"] = housing_cat_encoded.toarray()


In [154]:
cat_encoder = OneHotEncoder()
housing_cat_encoded  = cat_encoder.fit_transform(train_X[["Destination"]])
housing_cat_encoded.toarray()
train_X["Destination"] = housing_cat_encoded.toarray()

In [155]:
train_X["Deck"] = train_X["Cabin"].str[0]

train_X["Side"] = train_X["Cabin"].str[2]

train_X.drop(["Cabin", "Name"], axis=1, inplace=True)

cat_encoder = OneHotEncoder()
housing_cat_encoded  = cat_encoder.fit_transform(train_X[["Deck"]])
housing_cat_encoded.toarray()
train_X["Deck"] = housing_cat_encoded.toarray()

cat_encoder = OneHotEncoder()
housing_cat_encoded  = cat_encoder.fit_transform(train_X[["Side"]])
housing_cat_encoded.toarray()
train_X["Side"] = housing_cat_encoded.toarray()

train_X["Deck"]
train_X["Side"]

2333    0.0
2589    0.0
8302    0.0
8177    0.0
500     0.0
       ... 
5734    0.0
5191    0.0
5390    0.0
860     0.0
7270    0.0
Name: Side, Length: 6954, dtype: float64

In [156]:
train_X

,PassengerId,HomePlanet,CryoSleep,Destination,Age,VIP,RoomService,FoodCourt,ShoppingMall,Spa,VRDeck,relation,Deck,Side
2333,2513_01,1.0,False,0.0,28.0,False,0.0,55.0,0.0,656.0,0.0,1,0.0,0.0
2589,2774_02,1.0,False,0.0,17.0,False,0.0,1195.0,31.0,0.0,0.0,2,0.0,0.0
8302,8862_04,0.0,True,1.0,28.0,False,0.0,0.0,0.0,0.0,0.0,4,0.0,0.0
8177,8736_02,0.0,False,0.0,20.0,False,NaN,2.0,289.0,976.0,0.0,2,0.0,0.0
500,0539_02,0.0,True,1.0,36.0,False,0.0,0.0,0.0,0.0,0.0,2,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5734,6076_01,1.0,NaN,0.0,18.0,False,14.0,2.0,144.0,610.0,0.0,1,0.0,0.0
5191,5537_01,0.0,False,0.0,50.0,NaN,690.0,0.0,30.0,762.0,428.0,1,0.0,0.0
5390,5756_06,1.0,False,0.0,22.0,False,158.0,0.0,476.0,0.0,26.0,6,0.0,0.0
860,0925_01,0.0,False,0.0,34.0,False,379.0,0.0,1626.0,0.0,0.0,1,0.0,0.0


In [157]:
# train_X = np.asarray(train_X).astype('float32')
train_X

,PassengerId,HomePlanet,CryoSleep,Destination,Age,VIP,RoomService,FoodCourt,ShoppingMall,Spa,VRDeck,relation,Deck,Side
2333,2513_01,1.0,False,0.0,28.0,False,0.0,55.0,0.0,656.0,0.0,1,0.0,0.0
2589,2774_02,1.0,False,0.0,17.0,False,0.0,1195.0,31.0,0.0,0.0,2,0.0,0.0
8302,8862_04,0.0,True,1.0,28.0,False,0.0,0.0,0.0,0.0,0.0,4,0.0,0.0
8177,8736_02,0.0,False,0.0,20.0,False,NaN,2.0,289.0,976.0,0.0,2,0.0,0.0
500,0539_02,0.0,True,1.0,36.0,False,0.0,0.0,0.0,0.0,0.0,2,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5734,6076_01,1.0,NaN,0.0,18.0,False,14.0,2.0,144.0,610.0,0.0,1,0.0,0.0
5191,5537_01,0.0,False,0.0,50.0,NaN,690.0,0.0,30.0,762.0,428.0,1,0.0,0.0
5390,5756_06,1.0,False,0.0,22.0,False,158.0,0.0,476.0,0.0,26.0,6,0.0,0.0
860,0925_01,0.0,False,0.0,34.0,False,379.0,0.0,1626.0,0.0,0.0,1,0.0,0.0


In [158]:
# model = keras.Sequential([
#     keras.layers.Dense(10, activation='relu'),
#     keras.layers.Dense(1, activation='sigmoid')
# ])

# model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

In [159]:
# model.fit(train_X, train_y, validation_data=(valid_X, valid_y))